In [20]:
import numpy as np
import os
from importlib import import_module
import time

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
import data_loader

In [21]:
n_workers=1  #data loader workers
#n_gpu=3
start_lr=0.01
weight_decay=1e-4
nb_epochs=30
save_freq=1

In [22]:
kernel_size=7
#data_dir='/home/wyc/Desktop/preprocessed_data'
data_dir='C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data'
win_width=32  #label window, input window=win_width+kernel_size-1
batch_size=16  #256=32*8=256*1
model_path='conv'  
save_dir='C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\model_save'  #model save
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [23]:
model = import_module(model_path)
net, loss= model.get_model()

# net = net.cuda()
# loss = loss.cuda()
# cudnn.benchmark = True
net = DataParallel(net) 

In [24]:
loss

Loss(
  (classify_loss): BCEWithLogitsLoss()
)

In [25]:
dataset=data_loader.data_loader(data_dir,win_width, kernel_size,overlap=True,phase='train')
train_loader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = True,
        num_workers = n_workers,
        pin_memory=True)   #train/val pin_memory=True, test pin_memory=False

dataset=data_loader.data_loader(data_dir, win_width, kernel_size,overlap=True,phase='val')
val_loader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = False,
        num_workers = n_workers,
        pin_memory=True)   #train/val pin_memory=True, test pin_memory=False

trainfilelist:
['C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\train\\schumm-1_CQT.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\train\\schumm-1_label.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\train\\scn15_13_CQT.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\train\\scn15_13_label.npy']
traindata loaded
trainlabel loaded
valfilelist:
['C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\val\\chpn_op35_1_CQT.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\val\\chpn_op35_1_label.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\val\\scn16_2_CQT.npy', 'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data\\val\\scn16_2_label.npy']
valdata loaded
vallabel loaded


In [26]:
data_dir

'C:\\Users\\SUMIT\\Desktop\\CNN-LSTM_for_AMT\\preprocessed_data'

In [27]:
optimizer = optim.SGD(
        net.parameters(),
        start_lr,
        momentum = 0.9,
        weight_decay = weight_decay)

In [28]:
def get_lr(epoch,nb_epochs,start_lr):
    if epoch <= nb_epochs * 0.5:
        lr = start_lr
    elif epoch <= nb_epochs * 0.8:
        lr = 0.1 * start_lr
    else:
        lr = 0.01 * start_lr
    return lr

In [29]:



def train(data_loader, net, loss, epoch, optimizer, get_lr, save_freq, save_dir,nb_epochs,start_lr):
    start_time = time.time()

    net.train()
    lr = get_lr(epoch,nb_epochs,start_lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    metrics = []

    for i, (data, target) in enumerate(data_loader):
        data = Variable(data)
        target = Variable(target)

        output = net(data)
        loss_output = loss(output,target)#(8L, 88L, 32L, 1L)/(8L, 1L, 32L, 88L)
        #print loss_output[0].shape
        optimizer.zero_grad()
        loss_output[0].backward()
        optimizer.step()

        loss_output[0] = loss_output[0].data[0]
        metrics.append(loss_output)

    if epoch % save_freq == 0:
        state_dict = net.module.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].cpu()

        torch.save({
            'epoch': epoch,
            'save_dir': save_dir,
            'state_dict': state_dict
            },
            os.path.join(save_dir, '%03d.ckpt' % epoch))

    end_time = time.time()
    metrics = np.asarray(metrics, np.float32)

    print('Epoch %03d (lr %.5f),time %3.2f' % (epoch, lr,end_time - start_time))  # Framewise and notewise Accuracy precision,recall,F-score
    TP=np.sum(metrics[:, 1])
    Precision=TP/np.sum(metrics[:, 2])
    Recall=TP/np.sum(metrics[:, 3])
    Fscore=2*Precision*Recall/(Precision+Recall)
    print('Train:　loss %2.4f, Framewise Precision %3.2f,Recall %3.2f, F-score %3.2f' % (np.mean(metrics[:,0]),Precision,Recall,Fscore))
    print


def validate(data_loader, net, loss):
    start_time = time.time()

    net.eval()

    metrics = []
    for i, (data, target) in enumerate(data_loader):
        data = Variable(data, volatile=True)
        target = Variable(target, volatile=True)

        output = net(data)
        loss_output = loss(output, target, train=False)

        loss_output[0] = loss_output[0].data[0]
        metrics.append(loss_output)
    end_time = time.time()

    metrics = np.asarray(metrics, np.float32)
    TP=np.sum(metrics[:, 1])
    Precision=TP/np.sum(metrics[:, 2])
    Recall=TP/np.sum(metrics[:, 3])
    Fscore=2*Precision*Recall/(Precision+Recall)
    print('Validation: Loss %2.4f,Framewise Precision %3.2f,Recall %3.2f, F-score %3.2f' % (np.mean(metrics[:,0]),Precision,Recall,Fscore))
    print
    print



if __name__=='__main__':
    start_epoch=0
    for epoch in range(start_epoch,nb_epochs):
        train(train_loader, net, loss, epoch, optimizer, get_lr, save_freq, save_dir,nb_epochs,start_lr)
        validate(val_loader, net, loss)

C:\Users\SUMIT\Desktop\CNN-LSTM_for_AMT\conv.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return [loss,TP.data[0], pos_precision.data[0], pos_recall.data[0]] #F-score must be computed by whole epoch
C:\Users\SUMIT\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 000 (lr 0.01000),time 49.95
Train:　loss 0.5026, Framewise Precision 0.03,Recall 0.23, F-score 0.05


C:\Users\SUMIT\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\SUMIT\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:60: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\SUMIT\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:65: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[[  1.01723500e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  9.84564796e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.82189137e-01   0.00000000e+00   0.00000000e+00   5.80000000e+01]
 [  1.71917498e-01   0.00000000e+00   0.00000000e+00   2.64000000e+02]
 [  1.88590571e-01   0.00000000e+00   0.00000000e+00   5.42000000e+02]
 [  1.90283865e-01   0.00000000e+00   0.00000000e+00   5.00000000e+02]
 [  1.80639029e-01   0.00000000e+00   0.00000000e+00   5.54000000e+02]
 [  2.09243685e-01   0.00000000e+00   0.00000000e+00   9.29000000e+02]
 [  1.81095466e-01   0.00000000e+00   0.00000000e+00   1.59400000e+03]
 [  1.70951799e-01   0.00000000e+00   0.00000000e+00   1.55700000e+03]
 [  1.71663344e-01   0.00000000e+00   0.00000000e+00   1.43500000e+03]
 [  1.84558168e-01   0.00000000e+00   0.00000000e+00   1.59600000e+03]
 [  1.87386572e-01   0.00000000e+00   0.00000000e+00   1.23100000e+03]
 [  2.07368135e-01   0.00000000e+00   0.00000000e+00   1.50400000e+03]
 [  2.

C:\Users\SUMIT\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in float_scalars


Epoch 001 (lr 0.01000),time 53.02
Train:　loss 0.1006, Framewise Precision 0.34,Recall 0.01, F-score 0.02
[[  2.63695065e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.11229001e-02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.65543477e-02   0.00000000e+00   0.00000000e+00   5.80000000e+01]
 [  5.03204949e-02   0.00000000e+00   0.00000000e+00   2.64000000e+02]
 [  7.49275759e-02   0.00000000e+00   0.00000000e+00   5.42000000e+02]
 [  6.99110776e-02   0.00000000e+00   0.00000000e+00   5.00000000e+02]
 [  6.93625137e-02   0.00000000e+00   0.00000000e+00   5.54000000e+02]
 [  1.08333379e-01   0.00000000e+00   0.00000000e+00   9.29000000e+02]
 [  1.62759066e-01   0.00000000e+00   0.00000000e+00   1.59400000e+03]
 [  1.60928041e-01   0.00000000e+00   0.00000000e+00   1.55700000e+03]
 [  1.43767893e-01   0.00000000e+00   0.00000000e+00   1.43500000e+03]
 [  1.62199199e-01   0.00000000e+00   0.00000000e+00   1.59600000e+03]
 [  1.32274151e-01   0.00000000e+00   0.000

Epoch 002 (lr 0.01000),time 54.05
Train:　loss 0.0786, Framewise Precision 0.27,Recall 0.01, F-score 0.03


KeyboardInterrupt: 